In [ ]:
import torch
from diffusers import PixArtSigmaPipeline, PixArtAlphaPipeline
from pathlib import Path
import json
from diffusers.utils import make_image_grid

In [ ]:
# iterate over checkpoints
# MODELDIR = "pixart-fulltune_pixelart"
MODELDIR = "experiments/pixart_fulltune_animediese"
path = Path(MODELDIR)
print(path.exists())

# prompts = ["a portrait of a woman", 
#            "a portrait of a woman in pixelart style", 
#            "a portrait of a man", 
#            "a photo of a corgi dog", 
#            "an image of a husky"]

prompts = ["an anime drawing of a woman with long hair", 
           "anidiese a woman with long hair", 
           "anidiese The image features a woman with long hair",
        #    "anidiese The image features a female character",
           "anidiese The image features a female character with long hair.",
        #    "The image features a woman with long blue hair, wearing a red christmas-style uniform. She has a red headband with deer antlers.",
        #    "a woman with long blue hair, wearing a red christmas-style uniform. She has a red headband with deer antlers."
            "anidiese Husky dog."
           ]

ROOTNAME = "PixArt-alpha/PixArt-Sigma-XL-2-1024-MS"

In [ ]:
grids = []
divisibleby = 1000
size = 256
ckptdirs = list(path.glob("saved-*"))
print(ckptdirs)
pipeline = PixArtSigmaPipeline.from_pretrained(ROOTNAME).to(torch.device("cuda", 2))
pipeline.transformer = pipeline.transformer.from_pretrained("PixArt-alpha/PixArt-Sigma-XL-2-512-MS", subfolder="transformer").to(pipeline.device)

promptimages = []
for prompt in prompts:
    image = pipeline(prompt).images[0].resize((size,size))
    promptimages.append(image)
grid = make_image_grid(promptimages, 1, len(promptimages))
grids.append(grid)
display(grid)

for ckptpath in sorted(ckptdirs, key=lambda x: int(x.name[len("saved-"):])):
    number = int(ckptpath.name[len("saved-"):])
    if number % divisibleby == 0:
        print(ckptpath)
        # load unet from checkpoint
        pipeline.transformer = pipeline.transformer.from_pretrained(ckptpath).to(pipeline.device)
        
        promptimages = []
        for prompt in prompts:
            image = pipeline(prompt).images[0].resize((size,size))
            promptimages.append(image)
        grid = make_image_grid(promptimages, 1, len(promptimages))
        grids.append(grid)
            
        display(grid)

In [ ]:
display(make_image_grid(grids[1:], len(grids[1:]), 1))


In [ ]:
display(grids[0])